In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [9]:
%%time 
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

CPU times: user 9.33 ms, sys: 920 µs, total: 10.2 ms
Wall time: 11.7 ms


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.shape

In [ ]:
train_data.columns

In [ ]:
train_data.describe()

In [ ]:
train_data.Age.hist()

In [ ]:
import seaborn as sns


In [ ]:
sns.countplot(x="Sex", hue="Survived", data=train_data)

So to conclude from above plot:
* majority of male are NOT survived
* majority of female are survived
* male are greater than female


In [ ]:
sns.heatmap(data=train_data.corr(), annot=True )

Let\`s observe heatmap
* Low correlations are between Survived and [Fare & Pclass]
* Zero correlations are between Survived and [Age, SibSp, PassengerId and Parch]
* Age & Pclass has normal negative correlation which means that Pclass are devived into Age category
* Fare & Pclass also have normolized negative correlation which is probably means relationship
* Parch and SibSp have normal positive correlation value
* PassengerId does not have any high or low correlation with other feature -> so we drop PassenderId


Tose correlation results tell us that Zero correlation features should be eliminated.
However, Age i think must stay for modelling.

In [ ]:
train_data.tail()

In [11]:
df = train_data
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
sex_bin = {"female": 0,
           "male": 1}
df['Sex'] = df['Sex'].map(sex_bin)


In [ ]:
df.head()

In [ ]:
df.groupby(df.Age.isnull())["Name"].head()

In [ ]:
df.head()

In [ ]:
df.Pclass.unique()


In [ ]:
df.loc[(df.Age.isnull()) & (df.Embarked == 'S')].info()

In [ ]:
df.loc[(df.Age.isnull()) & (df.Embarked == 'Q')].info()

In [ ]:
df.loc[(df.Age.isnull()) & (df.Embarked == 'C')].info()

In [ ]:
df.Name[0].split()

In [ ]:
df.loc[( df.Name for i in range(len(df.Name)) if "Mr." == df.Name[i].split() )] 

* Age column has many missing values 
* Cabin column also has many missing values 
* Embarked column has 2 missing values
Now i should replace missing value to reallistic values:


In [13]:
y_train = df.Survived
X_train = df.drop("Survived", axis=1)


In [14]:
X_train = X_train.drop(["Ticket", "Cabin"], axis=1)

In [15]:
X_train = X_train.drop(["PassengerId"], axis=1)

In [ ]:
X_train.head()

In [ ]:
sns.countplot(x="Parch", hue="SibSp", data=X_train)

In [ ]:
X_train.columns

In [16]:

X_train.Age.fillna(X_train.Age.mean(), inplace=True)

In [17]:
X_train.Embarked.fillna("S", inplace=True)

In [ ]:
X_train.info()

In [18]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True)

In [ ]:
X.head()

In [19]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model',  RandomForestClassifier(n_estimators=100, max_depth=5, random_state=7))
                             ])







In [ ]:
scores = -1 * cross_val_score(my_pipeline, X, y_train,
                              cv=5,
                              scoring='neg_mean_absolute_error')

In [ ]:
my_pipeline.fit(X, y_train)

In [20]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
test.head()

In [21]:
my_test = test.drop(["PassengerId", "Name", "Cabin", "Ticket"],axis=1)

In [ ]:
test.info()

In [22]:
my_test.Age.fillna(test.Age.mean(), inplace=True)

In [23]:
my_test.Fare.fillna(test.Fare.mean(), inplace=True)

In [24]:
sex_bin = {"female": 0,
           "male": 1}
my_test['Sex'] = my_test['Sex'].map(sex_bin)

In [25]:
X_test = pd.get_dummies(my_test, drop_first=True)

In [26]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,1,0
1,3,0,47.0,1,0,7.0000,0,1
2,2,1,62.0,0,0,9.6875,1,0
3,3,1,27.0,0,0,8.6625,0,1
4,3,0,22.0,1,1,12.2875,0,1


Prediction is Done!

Big part of my task was replcaing missing values 
I must update Age and Ebarked coumns again with more clear algorythm of replacing missing values
I used mean() for Age 
for Embarked I chose "S" because it was the most popular 


In [7]:
from sklearn.ensemble import RandomForestClassifier 


In [40]:
model = RandomForestClassifier()


In [61]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
params = {"n_estimators": np.arange(50,700,50),
         "max_depth": np.arange(1,10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)


In [62]:
grid = GridSearchCV(estimator = model,
                   param_grid = params,
                   cv=skf,
                   n_jobs=-1,
                   verbose=1)
grid.fit(X, y_train)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   54.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 585 out of 585 | elapsed:  2.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=7, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650])},
             verbose=1)

In [63]:
grid.best_params_

{'max_depth': 6, 'n_estimators': 250}

In [58]:
grid.best_estimator_

RandomForestClassifier(max_depth=1, n_estimators=300)

In [64]:
grid.best_score_

0.8338773460548616

In [65]:
preds = grid.predict(X_test)

In [66]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score
